# Import libraries
import pandas as pd

# model libraries
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb 
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# cross validation and grid libraries 
from sklearn import metrics
from sklearn import cross_validation
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.grid_search import RandomizedSearchCV

# plotting libraries in case we need it 
from matplotlib import pyplot as plt 
%matplotlib inline


# Intro 
This notebook is about to create Blender models, which includes: 
- **Layer1** model selection with grid search 
    - ~~Holdout % optimization with grid~~ <- done previously (60-40)
- **Layer2** model selection with grid search 
    - Holdout % optimization with grid
    - for dataset including layer1 predictions only 
    - for dataset including layer1 predictions and original dataset as well
- **Test for layer3**, which is using predictions from layer2 as train set and a portion of the holdout set for test

** Models: ** 
- Logistic regression
- Nearest neighbours
- Random forest 
- Xgboost 
- Naive Bayes with bernoulli 
- Adaboost
- Extra trees classifier
- Support vector Classifier 
- Quadratic discriminance analysis



----------
# Layer1 grid search 

With layer-holdout 60-40% split

In [2]:
# Load and split data
train = pd.read_csv('Data/FE_v1_train.csv')
test = pd.read_csv('Data/FE_v1_test.csv')
train_y = pd.read_csv('Data/train_y.csv')

splitter = StratifiedShuffleSplit(y=train_y, n_iter=1, train_size=0.6, 
                                      test_size=0.4, random_state=42)

for train_index, holdout_index in splitter: 
    train_layer1 = train.iloc[train_index, :]
    train_layer1.reset_index(drop=True, inplace=True)
    train_y_layer1 = train_y.iloc[train_index, :]
    train_y_layer1.reset_index(drop=True, inplace=True)
    train_l1_holdout = train.iloc[holdout_index, :]
    train_l1_holdout.reset_index(drop=True, inplace=True)
    train_y_l1_holdout = train_y.iloc[holdout_index, :]
    train_y_l1_holdout.reset_index(drop=True, inplace=True)

In [9]:
# Determine model parameters: 
params_logreg = {'n_jobs':[4],
                 'C':[0.0001,0.1,0.5,1.5,2,5,10], 
                 'fit_intercept':[False,True], 
                 'max_iter':[10,100,500,1000,2000],
                 'solver':['newton-cg','lbfgs','liblinear','sag'],
                 'tol':[0.00001,0.0001,0.001,0.1,0.5],
                 'verbose':[0],
                 'random_state':[42]}

params_knn = {'n_jobs':[4],
              'n_neighbors':[5,50,100,500,1000,1500,2000,3000,4000,5000], 
              'p':[1,2,3,4,5,10,11,12,15,20],
              'leaf_size':[10,20,30,40,50,60],
              'algorithm':['auto','ball_tree','kd_tree','brute']}

params_rf = {'n_jobs':[4], 
             'criterion':['gini', 'entropy'],
             'n_estimators':[250,500,1000,1500,2000,3000,4000,5000], 
             'max_features':[1,5,10,15,20,25], 
             'max_depth':[1,5,10,15,20,25,50,100], 
             'min_samples_split':[1,5,10,25,50], 
             'min_samples_leaf':[1,5,10,25,50], 
             'oob_score':[True,False], 
             'verbose':[0], 
             'random_state':[42]}

params_xgb = {'silent':[1],
              'nthread':[4], 
              'seed':[42], 
              'max_depth':[1,5,10,15,20,25],
              'subsample':[0.2,0.5,0.7,1],
              'reg_lambda':[1,2,5,10],
              'learning_rate':[0.0001,0.001,0.01,0.1,0.3], 
              'gamma':[0,0.0001,0.001,0.01,0.1],
              'n_estimators':[250,500,1000,1500,2000,3000,4000,5000]
             }

params_naive = {'alpha':[0.1,0.5,0.75,1,1.5,2,5,10,15,25,30,40,50,60,70,80,90,100,200,500], 
                'binarize':[0.1,0.5,0.75,1,1.5,2,5,10,15,25,30,40,50,60,70,80,90,100,200,500],
                'fit_prior':[True, False], 
               }

params_ada = {'n_estimators':[250,500,1000,1500,2000,3000,4000,5000],
              'learning_rate':[0.0001,0.001,0.01,0.1,0.5,0.75,1.5,2,5,10,25,50], 
              'random_state':[42] 
             }

# params_extra = {'n_jobs':[4],
#                 'criterion':['gini', 'entropy'],
#                 'n_estimators':[250,500,1000,1500,2000,3000,4000,5000], 
#                 'max_features':[1,5,10,15,20,25], 
#                 'max_depth':[1,5,10,15,20,25,50,100], 
#                 'min_samples_split':[1,5,10,25,50], 
#                 'min_samples_leaf':[1,5,10,25,50], 
#                 'oob_score':[True,False], 
#                 'bootstrap':[True],
#                 'verbose':[0], 
#                 'random_state':[42]}

params_svc = {'C':[0.0001,0.1,0.5,1.5,2,5,10], 
              'kernel':['linear','poly','rbf','sigmoid'], 
              'degree':[1,3,5,10,15,20,25,50], 
              'gamma':[0.0001,0.001,0.01,0.1,0.5,0.75,1,1.5,1.75,2,5,10],
              'coef0':[0.0001,0.001,0.01,0.1,0.5,0.75,1,1.5,1.75,2,5,10],
              'probability':[True],
              'shrinking':[True, False],
              'tol':[0.0001,0.00001],
              'random_state':[42]
             }

In [4]:
# Determine models
model_logreg = LogisticRegression()
model_knn = KNeighborsClassifier()
model_rf = RandomForestClassifier()
model_xgb = xgb.XGBClassifier()
model_naive = BernoulliNB()
model_ada = AdaBoostClassifier()
model_extra = ExtraTreesClassifier()
model_svc = SVC()

In [5]:
#Logistic regression
grid_logreg = RandomizedSearchCV(n_iter = 500, estimator=model_logreg, param_distributions=params_logreg, 
                                 n_jobs=2, cv=5, refit=True, verbose=1, scoring='log_loss')

grid_logreg.fit(train_layer1, train_y_layer1.target)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   21.2s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:  1.5min
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:  3.9min
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:  6.7min
[Parallel(n_jobs=2)]: Done 1246 tasks      | elapsed: 10.5min
[Parallel(n_jobs=2)]: Done 1796 tasks      | elapsed: 14.8min
[Parallel(n_jobs=2)]: Done 2446 tasks      | elapsed: 20.1min
[Parallel(n_jobs=2)]: Done 2500 out of 2500 | elapsed: 20.6min finished
/home/ngergoo/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:1207: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  for (class_, warm_start_coef_) in zip(classes_, warm_start_coef))
/home/ngergoo/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:1207: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  for (class_, warm_start_coef_) in zip(classes_, warm_start_coef))
/home/ng

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=500, n_jobs=2,
          param_distributions={'C': [0.0001, 0.1, 0.5, 1.5, 2, 5, 10], 'n_jobs': [4], 'verbose': [0], 'tol': [1e-05, 0.0001, 0.001, 0.1, 0.5], 'fit_intercept': [False, True], 'random_state': [42], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag'], 'max_iter': [10, 100, 500, 1000, 2000]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='log_loss', verbose=1)

In [6]:
# Knn
grid_knn = RandomizedSearchCV(n_iter=10, estimator=model_knn, param_distributions=params_knn, n_jobs=2, cv=5, 
                             refit=True, verbose=1, scoring='log_loss')

grid_knn.fit(train_layer1, train_y_layer1.target)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 89.0min
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed: 117.5min finished
/home/ngergoo/anaconda2/lib/python2.7/site-packages/sklearn/metrics/pairwise.py:1060: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  for s in gen_even_slices(Y.shape[0], n_jobs))
/home/ngergoo/anaconda2/lib/python2.7/site-packages/sklearn/metrics/pairwise.py:1060: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  for s in gen_even_slices(Y.shape[0], n_jobs))


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
          fit_params={}, iid=True, n_iter=10, n_jobs=2,
          param_distributions={'n_neighbors': [5, 50, 100, 500, 1000, 1500, 2000, 3000, 4000, 5000], 'n_jobs': [4], 'leaf_size': [10, 20, 30, 40, 50, 60], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 'p': [1, 2, 3, 4, 5, 10, 11, 12, 15, 20]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='log_loss', verbose=1)

In [7]:
# Random Forest
grid_rf = RandomizedSearchCV(estimator=model_rf, param_distributions=params_rf, n_jobs=2, cv=5, refit=True, 
                             verbose=1, scoring='log_loss')

grid_rf.fit(train_layer1, train_y_layer1.target)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 275.5min
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed: 291.9min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params={}, iid=True, n_iter=10, n_jobs=2,
          param_distributions={'n_estimators': [250, 500, 1000, 1500, 2000, 3000, 4000, 5000], 'min_samples_split': [1, 5, 10, 25, 50], 'oob_score': [True, False], 'n_jobs': [4], 'criterion': ['gini', 'entropy'], 'verbose': [0], 'max_features': [1, 5, 10, 15, 20, 25], 'random_state': [42], 'max_depth': [1, 5, 10, 15, 20, 25, 50, 100], 'min_samples_leaf': [1, 5, 10, 25, 50]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='log_loss', verbose=1)

In [7]:
# Bernoulli Bayes
grid_naive = RandomizedSearchCV(n_iter=40, estimator=model_naive, param_distributions=params_naive, n_jobs=8, 
                                cv=5, refit=True, verbose=1, scoring='log_loss')
grid_naive.fit(train_layer1, train_y_layer1.target)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=8)]: Done  52 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    3.7s finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True),
          fit_params={}, iid=True, n_iter=40, n_jobs=8,
          param_distributions={'binarize': [0.1, 0.5, 0.75, 1, 1.5, 2, 5, 10, 15, 25, 30, 40, 50, 60, 70, 80, 90, 100, 200, 500], 'alpha': [0.1, 0.5, 0.75, 1, 1.5, 2, 5, 10, 15, 25, 30, 40, 50, 60, 70, 80, 90, 100, 200, 500], 'fit_prior': [True, False]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='log_loss', verbose=1)

In [ ]:
# Adaboost
grid_ada = RandomizedSearchCV(n_iter=15, estimator=model_ada, param_distributions=params_ada, n_jobs=8, 
                              cv=5, refit=True, verbose=1, scoring='log_loss')
grid_ada.fit(train_layer1, train_y_layer1.target)

In [ ]:
# XGB
splitter = StratifiedShuffleSplit(y=train_y_layer1, n_iter=1, train_size=0.7, 
                                      test_size=0.3, random_state=42)

for train_index, test_index in splitter: 
    train_l1_1 = train_layer1.iloc[train_index,:]
    train_l1_1.reset_index(drop=True, inplace=True)
    train_y_l1_1 = train_y_layer1.iloc[train_index,:]
    train_y_l1_1.reset_index(drop=True, inplace=True)
    train_l1_2 = train_layer1.iloc[test_index,:]
    train_l1_2.reset_index(drop=True, inplace=True)
    train_y_l1_2 = train_y_layer1.iloc[test_index,:]
    train_l1_2.reset_index(drop=True, inplace=True)


xgb_fit_params = {'eval_set':[(train_l1_1, train_y_l1_1),(train_l1_2, train_y_l1_2)], 
                  'early_stopping_rounds':25, 'verbose':0}

grid_xgb = RandomizedSearchCV(n_iter=20, estimator=model_xgb, param_distributions=params_xgb, 
                              n_jobs=2, cv=5, refit=True, verbose=1, scoring='log_loss', 
                              fit_params=xgb_fit_params
                             )

grid_xgb.fit(train_layer1, train_y_layer1.target)

In [12]:
# these are results from bigdell rerun
print 'Logreg: ', grid_logreg.best_params_, grid_logreg.best_score_
print 'KNN: ', grid_knn.best_params_, grid_knn.best_score_
# print 'RandomForest: ', grid_rf.best_params_, grid_rf.best_score_
print 'Bernoulli bayes: ', grid_naive.best_params_, grid_naive.best_score_
# print 'Adaboost: ', grid_ada.best_params_, grid_ada.best_score_
# print 'Xgboost: ', grid_xgb.best_params_, grid_xgb.best_score_

 Logreg:  {'fit_intercept': False, 'C': 0.1, 'n_jobs': 4, 'verbose': 0, 'solver': 'sag', 'max_iter': 1000, 'random_state': 42, 'tol': 0.001} -0.691717645023
KNN:  {'p': 2, 'n_jobs': 4, 'leaf_size': 20, 'algorithm': 'ball_tree', 'n_neighbors': 1000} -0.692166108114
Bernoulli bayes:  {'binarize': 10, 'alpha': 1.5, 'fit_prior': True} -0.692666541942


In [13]:
# these are results from littleDell
print 'Logreg: ', grid_logreg.best_params_, grid_logreg.best_score_
print 'KNN: ', grid_knn.best_params_, grid_knn.best_score_
print 'RandomForest: ', grid_rf.best_params_, grid_rf.best_score_
print 'Bernoulli bayes: ', grid_naive.best_params_, grid_naive.best_score_
print 'Adaboost: ', grid_ada.best_params_, grid_ada.best_score_
print 'Xgboost: ', grid_xgb.best_params_, grid_xgb.best_score_

 Logreg:  {'fit_intercept': False, 'C': 0.1, 'n_jobs': 4, 'verbose': 0, 'solver': 'sag', 'max_iter': 100, 'random_state': 42, 'tol': 0.001} -0.691717645023
KNN:  {'p': 1, 'n_jobs': 4, 'leaf_size': 20, 'algorithm': 'ball_tree', 'n_neighbors': 4000} -0.692126133875
RandomForest:  {'oob_score': False, 'n_jobs': 4, 'verbose': 0, 'min_samples_leaf': 25, 'n_estimators': 1500, 'max_features': 15, 'random_state': 42, 'criterion': 'entropy', 'min_samples_split': 50, 'max_depth': 10} -0.692124787781
Bernoulli bayes:  {'binarize': 15, 'alpha': 0.5, 'fit_prior': True} -0.693057262966
Adaboost:  {'n_estimators': 5000, 'learning_rate': 0.0001, 'random_state': 42} -0.692654655439
Xgboost:  {'silent': 1, 'learning_rate': 0.1, 'nthread': 4, 'n_estimators': 5000, 'subsample': 0.7, 'reg_lambda': 5, 'seed': 42, 'max_depth': 1, 'gamma': 0.0001} -0.691804040637


In [ ]:
# print out results into GridResults/

In [ ]:
# I had some problems with the following algos: 

# Extra trees - parameter error maybe?
grid_extra = RandomizedSearchCV(estimator=model_extra, param_distributions=params_extra, n_jobs=1, cv=5,
                               refit=True, verbose=1, scoring='log_loss')
grid_extra.fit(train_layer1, train_y_layer1.target)

In [ ]:
# support vector classifier - takes too much time, on bigdell one night is not enough
grid_svc = RandomizedSearchCV(estimator=model_svc, param_distributions=params_svc, n_jobs=8, cv=5, 
                             refit=True, verbose=1, scoring='log_loss')

grid_svc.fit(train_layer1, train_y_layer1.target)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


# Blend_v1
## Create predictions with layer1 models on holdout set and test set

In [22]:
# Set model parameters
params_logreg = {'n_jobs':4,
                 'C':0.1, 
                 'fit_intercept':False, 
                 'max_iter':100,
                 'solver':'sag',
                 'tol':0.001,
                 'verbose':0,
                 'random_state':42
                }

params_knn = {'n_jobs':4,
              'n_neighbors':4000, 
              'p':1,
              'leaf_size':20,
              'algorithm':'ball_tree'
             }

params_rf = {'n_jobs':4, 
             'criterion':'entropy',
             'n_estimators':1500, 
             'max_features':15, 
             'max_depth':10,
             'min_samples_split':50, 
             'min_samples_leaf':25, 
             'oob_score':False, 
             'verbose':0,
             'random_state':42
            }

params_xgb = {'silent':1,
              'nthread':4, 
              'seed':42, 
              'max_depth':1,
              'subsample':0.7,
              'reg_lambda':5,
              'learning_rate':0.1, 
              'gamma':0.0001,
              'n_estimators':5000
             }

params_naive = {'alpha':0.5,
                'binarize':15,
                'fit_prior':True 
               }

params_ada = {'n_estimators':5000,
              'learning_rate':0.0001, 
              'random_state':42 
             }

# Set models
model_logreg = LogisticRegression(**params_logreg)
model_knn = KNeighborsClassifier(**params_knn)
model_rf = RandomForestClassifier(**params_rf)
model_xgb = xgb.XGBClassifier(**params_xgb)
model_naive = BernoulliNB(**params_naive)
model_ada = AdaBoostClassifier(**params_ada)
# model_extra = ExtraTreesClassifier()
# model_svc = SVC() -> takes too much time for randomGrid

In [25]:
# create predictions for holdout set
model_logreg.fit(train_layer1, train_y_layer1.target)
print 'logreg done...'
model_knn.fit(train_layer1, train_y_layer1.target)
print 'knn done...'
model_rf.fit(train_layer1, train_y_layer1.target)
print 'rf done...'
model_naive.fit(train_layer1, train_y_layer1.target)
print 'naive done...'
model_ada.fit(train_layer1, train_y_layer1.target)
print 'ada done...'

logreg done...
knn done...
rf done...
naive done...
ada done...


In [26]:
#xgboost
# XGB
splitter = StratifiedShuffleSplit(y=train_y_layer1, n_iter=1, train_size=0.7, 
                                      test_size=0.3, random_state=42)

for train_index, test_index in splitter: 
    train_l1_1 = train_layer1.iloc[train_index,:]
    train_l1_1.reset_index(drop=True, inplace=True)
    train_y_l1_1 = train_y_layer1.iloc[train_index,:]
    train_y_l1_1.reset_index(drop=True, inplace=True)
    train_l1_2 = train_layer1.iloc[test_index,:]
    train_l1_2.reset_index(drop=True, inplace=True)
    train_y_l1_2 = train_y_layer1.iloc[test_index,:]
    train_l1_2.reset_index(drop=True, inplace=True)

model_xgb.fit(train_layer1, train_y_layer1.target,
            eval_set=[(train_l1_1, train_y_l1_1),(train_l1_2, train_y_l1_2)], 
            early_stopping_rounds=25, verbose=0)
print 'xgb done...'

xgb done...


In [27]:
# Predict on holdout set
holdout_predict_logreg = model_logreg.predict_proba(train_l1_holdout)
holdout_predict_knn = model_knn.predict_proba(train_l1_holdout)
holdout_predict_rf = model_rf.predict_proba(train_l1_holdout)
holdout_predict_naive = model_naive.predict_proba(train_l1_holdout)
holdout_predict_ada = model_ada.predict_proba(train_l1_holdout)
holdout_predict_xgb = model_xgb.predict_proba(train_l1_holdout)

In [30]:
# Calculate error on holdout set
print 'logloss', metrics.log_loss(y_true=train_y_l1_holdout, y_pred=holdout_predict_logreg[:,1])
print 'knn', metrics.log_loss(y_true=train_y_l1_holdout, y_pred=holdout_predict_knn[:,1])
print 'rf', metrics.log_loss(y_true=train_y_l1_holdout, y_pred=holdout_predict_rf[:,1])
print 'naive', metrics.log_loss(y_true=train_y_l1_holdout, y_pred=holdout_predict_naive[:,1])
print 'ada', metrics.log_loss(y_true=train_y_l1_holdout, y_pred=holdout_predict_ada[:,1])
print 'xgb', metrics.log_loss(y_true=train_y_l1_holdout, y_pred=holdout_predict_xgb[:,1])

logloss 0.691567060898
knn 0.691726372791
rf 0.691702025755
naive 0.693022286855
ada 0.692435168301
xgb 0.691709935212


Naive bayes and adaboost have a poor performance, the others are ok

In [34]:
# Predict on test set
test_predict_logreg = model_logreg.predict_proba(test.iloc[:,1])
test_predict_knn = model_knn.predict_proba(test.iloc[:,1])
test_predict_rf = model_rf.predict_proba(test.iloc[:,1])
test_predict_naive = model_naive.predict_proba(test.iloc[:,1])
test_predict_ada = model_ada.predict_proba(test.iloc[:,1])
test_predict_xgb = model_xgb.predict_proba(test.iloc[:,1])

In [58]:
# concat holdout with predictions 
temp = pd.concat([pd.DataFrame(holdout_predict_logreg[:,1]),
                 pd.DataFrame(holdout_predict_knn[:,1]), 
                 pd.DataFrame(holdout_predict_rf[:,1]),
                 pd.DataFrame(holdout_predict_naive[:,1]),
                 pd.DataFrame(holdout_predict_ada[:,1]), 
                 pd.DataFrame(holdout_predict_xgb[:,1])], axis=1)
temp.columns = ['pred_logreg', 'pred_knn', 'pred_rf', 'pred_naive', 'pred_ada', 'pred_xgb']

train_l1_holdout_blend_v1 = pd.concat([train_l1_holdout, temp], axis=1)


# concat test with predictions
temp = pd.concat([pd.DataFrame(test_predict_logreg[:,1]),
                 pd.DataFrame(test_predict_knn[:,1]), 
                 pd.DataFrame(test_predict_rf[:,1]),
                 pd.DataFrame(test_predict_naive[:,1]),
                 pd.DataFrame(test_predict_ada[:,1]), 
                 pd.DataFrame(test_predict_xgb[:,1])], axis=1)
temp.columns = ['pred_logreg', 'pred_knn', 'pred_rf', 'pred_naive', 'pred_ada', 'pred_xgb']

test_blend_v1 = pd.concat([test, temp], axis=1)

In [60]:
# save tables in csv files
pd.DataFrame.to_csv(train_layer1, 'Blend_data/Blend_v1_train_layer1.csv', index=False)
pd.DataFrame.to_csv(train_y_layer1, 'Blend_data/Blend_v1_train_y_layer1.csv', index=False)
pd.DataFrame.to_csv(train_l1_holdout_blend_v1, 'Blend_data/Blend_v1_train_l1_holdout.csv', index=False)
pd.DataFrame.to_csv(train_y_l1_holdout, 'Blend_data/Blend_v1_train_y_holdout.csv', index=False)

pd.DataFrame.to_csv(test_blend_v1, 'Blend_data/Blend_v1_test.csv', index=False)

## Grid search for blender model WITH original data and predictions

In [3]:
train_l1_holdout_blend_v1 = pd.read_csv('Blend_data/Blend_v1_train_l1_holdout.csv')
train_y_l1_holdout = pd.read_csv('Blend_data/Blend_v1_train_y_holdout.csv')

test_blend_v1 = pd.read_csv('Blend_data/Blend_v1_test.csv')

In [4]:
# Determine model parameters: 
params_logreg = {'n_jobs':[4],
                 'C':[0.0001,0.1,0.5,1.5,2,5,10], 
                 'fit_intercept':[False,True], 
                 'max_iter':[10,100,500,1000,2000],
                 'solver':['newton-cg','lbfgs','liblinear','sag'],
                 'tol':[0.00001,0.0001,0.001,0.1,0.5],
                 'verbose':[0],
                 'random_state':[42]}

params_knn = {'n_jobs':[4],
              'n_neighbors':[5,50,100,500,1000,1500,2000,3000,4000,5000], 
              'p':[1,2,3,4,5,10,11,12,15,20],
              'leaf_size':[10,20,30,40,50,60],
              'algorithm':['auto','ball_tree','kd_tree','brute']}

params_rf = {'n_jobs':[4], 
             'criterion':['gini', 'entropy'],
             'n_estimators':[250,500,1000,1500,2000,3000,4000,5000], 
             'max_features':[1,5,10,15,20,25], 
             'max_depth':[1,5,10,15,20,25,50,100], 
             'min_samples_split':[1,5,10,25,50], 
             'min_samples_leaf':[1,5,10,25,50], 
             'oob_score':[True,False], 
             'verbose':[0], 
             'random_state':[42]}

params_xgb = {'silent':[1],
              'nthread':[4], 
              'seed':[42], 
              'max_depth':[1,5,10,15,20,25],
              'subsample':[0.2,0.5,0.7,1],
              'reg_lambda':[1,2,5,10],
              'learning_rate':[0.0001,0.001,0.01,0.1,0.3], 
              'gamma':[0,0.0001,0.001,0.01,0.1],
              'n_estimators':[250,500,1000,1500,2000,3000,4000,5000]
             }

params_naive = {'alpha':[0.1,0.5,0.75,1,1.5,2,5,10,15,25,30,40,50,60,70,80,90,100,200,500], 
                'binarize':[0.1,0.5,0.75,1,1.5,2,5,10,15,25,30,40,50,60,70,80,90,100,200,500],
                'fit_prior':[True, False], 
               }

params_ada = {'n_estimators':[250,500,1000,1500,2000,3000,4000,5000],
              'learning_rate':[0.0001,0.001,0.01,0.1,0.5,0.75,1.5,2,5,10,25,50], 
              'random_state':[42] 
             }

In [5]:
# Determine models
model_logreg = LogisticRegression()
model_knn = KNeighborsClassifier()
model_rf = RandomForestClassifier()
model_xgb = xgb.XGBClassifier()
model_naive = BernoulliNB()
model_ada = AdaBoostClassifier()

In [8]:
#Logistic regression
grid_logreg = RandomizedSearchCV(n_iter = 500, estimator=model_logreg, param_distributions=params_logreg, 
                                 n_jobs=2, cv=5, refit=True, verbose=1, scoring='log_loss')

grid_logreg.fit(train_l1_holdout_blend_v1, train_y_l1_holdout.target)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   30.2s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:  1.8min
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:  5.2min
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed: 10.8min
[Parallel(n_jobs=2)]: Done 1246 tasks      | elapsed: 17.0min
[Parallel(n_jobs=2)]: Done 1796 tasks      | elapsed: 23.3min
[Parallel(n_jobs=2)]: Done 2446 tasks      | elapsed: 32.5min
[Parallel(n_jobs=2)]: Done 2500 out of 2500 | elapsed: 33.8min finished
/home/ngergoo/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/ngergoo/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/sag.py:267: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/ngergoo/anaconda2/lib/python2.7/site-packages/skle

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=500, n_jobs=2,
          param_distributions={'C': [0.0001, 0.1, 0.5, 1.5, 2, 5, 10], 'n_jobs': [4], 'verbose': [0], 'tol': [1e-05, 0.0001, 0.001, 0.1, 0.5], 'fit_intercept': [False, True], 'random_state': [42], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag'], 'max_iter': [10, 100, 500, 1000, 2000]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='log_loss', verbose=1)

In [9]:
# Knn
grid_knn = RandomizedSearchCV(n_iter=10, estimator=model_knn, param_distributions=params_knn, n_jobs=2, cv=5, 
                             refit=True, verbose=1, scoring='log_loss')

grid_knn.fit(train_l1_holdout_blend_v1, train_y_l1_holdout.target)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 35.1min
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed: 38.2min finished
/home/ngergoo/anaconda2/lib/python2.7/site-packages/sklearn/metrics/pairwise.py:1060: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  for s in gen_even_slices(Y.shape[0], n_jobs))
/home/ngergoo/anaconda2/lib/python2.7/site-packages/sklearn/metrics/pairwise.py:1060: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  for s in gen_even_slices(Y.shape[0], n_jobs))


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
          fit_params={}, iid=True, n_iter=10, n_jobs=2,
          param_distributions={'n_neighbors': [5, 50, 100, 500, 1000, 1500, 2000, 3000, 4000, 5000], 'n_jobs': [4], 'leaf_size': [10, 20, 30, 40, 50, 60], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 'p': [1, 2, 3, 4, 5, 10, 11, 12, 15, 20]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='log_loss', verbose=1)

In [10]:
# Random Forest
grid_rf = RandomizedSearchCV(estimator=model_rf, param_distributions=params_rf, n_jobs=2, cv=5, refit=True, 
                             verbose=1, scoring='log_loss')

grid_rf.fit(train_l1_holdout_blend_v1, train_y_l1_holdout.target)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 115.9min
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed: 123.9min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params={}, iid=True, n_iter=10, n_jobs=2,
          param_distributions={'n_estimators': [250, 500, 1000, 1500, 2000, 3000, 4000, 5000], 'min_samples_split': [1, 5, 10, 25, 50], 'oob_score': [True, False], 'n_jobs': [4], 'criterion': ['gini', 'entropy'], 'verbose': [0], 'max_features': [1, 5, 10, 15, 20, 25], 'random_state': [42], 'max_depth': [1, 5, 10, 15, 20, 25, 50, 100], 'min_samples_leaf': [1, 5, 10, 25, 50]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='log_loss', verbose=1)

In [11]:
# Bernoulli Bayes
grid_naive = RandomizedSearchCV(n_iter=40, estimator=model_naive, param_distributions=params_naive, n_jobs=8, 
                                cv=5, refit=True, verbose=1, scoring='log_loss')
grid_naive.fit(train_l1_holdout_blend_v1, train_y_l1_holdout.target)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=8)]: Done  88 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    2.5s finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True),
          fit_params={}, iid=True, n_iter=40, n_jobs=8,
          param_distributions={'binarize': [0.1, 0.5, 0.75, 1, 1.5, 2, 5, 10, 15, 25, 30, 40, 50, 60, 70, 80, 90, 100, 200, 500], 'alpha': [0.1, 0.5, 0.75, 1, 1.5, 2, 5, 10, 15, 25, 30, 40, 50, 60, 70, 80, 90, 100, 200, 500], 'fit_prior': [True, False]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='log_loss', verbose=1)

In [12]:
# Adaboost - gives back some weird value error, about nans/infinite values in df X?!
grid_ada = RandomizedSearchCV(n_iter=10, estimator=model_ada, param_distributions=params_ada, n_jobs=8, 
                              cv=5, refit=True, verbose=1, scoring='log_loss')
grid_ada.fit(train_l1_holdout_blend_v1, train_y_l1_holdout.target)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed: 31.9min


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/ngergoo/anaconda2/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    157     pkg_name = mod_name.rpartition('.')[0]
    158     main_globals = sys.modules["__main__"].__dict__
    159     if alter_argv:
    160         sys.argv[0] = fname
    161     return _run_code(code, main_globals, None,
--> 162                      "__main__", fname, loader, pkg_name)
        fname = '/home/ngergoo/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    163 
    164 def run_module(mod_name, init_globals=None,
    165                run_name=None, alter_sys=False):
    166     """Execute a module's code without importing it

.............................

In [7]:
# XGB
splitter = StratifiedShuffleSplit(y=train_y_l1_holdout, n_iter=1, train_size=0.7, 
                                      test_size=0.3, random_state=42)

for train_index, test_index in splitter: 
    train_1 = train_l1_holdout_blend_v1.iloc[train_index,:]
    train_1.reset_index(drop=True, inplace=True)
    train_y_1 = train_y_l1_holdout.iloc[train_index,:]
    train_y_1.reset_index(drop=True, inplace=True)
    train_2 = train_l1_holdout_blend_v1.iloc[test_index,:]
    train_2.reset_index(drop=True, inplace=True)
    train_y_2 = train_y_l1_holdout.iloc[test_index,:]
    train_y_2.reset_index(drop=True, inplace=True)


xgb_fit_params = {'eval_set':[(train_1, train_y_1.target),(train_2, train_y_2.target)], 
                  'early_stopping_rounds':25, 'verbose':0}

grid_xgb = RandomizedSearchCV(n_iter=20, estimator=model_xgb, param_distributions=params_xgb, 
                              n_jobs=2, cv=5, refit=True, verbose=1, scoring='log_loss', 
                              fit_params=xgb_fit_params
                             )

grid_xgb.fit(train_l1_holdout_blend_v1, train_y_l1_holdout.target)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 10.1min
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed: 19.9min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
          fit_params={'eval_set': [(       feature1  feature2  feature3  feature4  feature5  feature6  feature7  \
0      0.695981  0.596473  0.515205  0.884092  0.320731  0.546269  0.095469
1      0.676988  0.644239  0.444708  0.558028  0.221347  0.092506  0.229707
2      0.387228  0.662142  0.639957  ...    1
11557    1
11558    0
Name: target, dtype: int64)], 'early_stopping_rounds': 25, 'verbose': 0},
          iid=True, n_iter=20, n_jobs=2,
          param_distributions={'n_estimators': [250, 500, 1000, 1500, 2000, 3000, 4000, 5000], 'subsample': [0.2, 0.5, 0.7, 1], 'reg_lambda': [1, 2, 5,

In [13]:
# results
print 'Logreg: ', grid_logreg.best_params_, grid_logreg.best_score_
print 'KNN: ', grid_knn.best_params_, grid_knn.best_score_
print 'RandomForest: ', grid_rf.best_params_, grid_rf.best_score_
print 'Bernoulli bayes: ', grid_naive.best_params_, grid_naive.best_score_
# print 'Adaboost: ', grid_ada.best_params_, grid_ada.best_score_
print 'Xgboost: ', grid_xgb.best_params_, grid_xgb.best_score_

Logreg:  {'fit_intercept': True, 'C': 0.1, 'n_jobs': 4, 'verbose': 0, 'solver': 'lbfgs', 'max_iter': 500, 'random_state': 42, 'tol': 0.001} -0.69152967219
KNN:  {'p': 1, 'n_jobs': 4, 'leaf_size': 40, 'algorithm': 'brute', 'n_neighbors': 3000} -0.69168230852
RandomForest:  {'oob_score': True, 'n_jobs': 4, 'verbose': 0, 'min_samples_leaf': 25, 'n_estimators': 4000, 'max_features': 5, 'random_state': 42, 'criterion': 'gini', 'min_samples_split': 1, 'max_depth': 10} -0.691457346439
Bernoulli bayes:  {'binarize': 10, 'alpha': 80, 'fit_prior': False} -0.692724946458
Xgboost:  {'silent': 1, 'learning_rate': 0.1, 'nthread': 4, 'n_estimators': 2000, 'subsample': 1, 'reg_lambda': 10, 'seed': 42, 'max_depth': 1, 'gamma': 0} -0.691545945181


In [20]:
pd.DataFrame.to_csv(pd.DataFrame(grid_logreg.grid_scores_), 'GridResults/blender_logreg.csv', index=False)
pd.DataFrame.to_csv(pd.DataFrame(grid_knn.grid_scores_), 'GridResults/blender_knn.csv', index=False)
pd.DataFrame.to_csv(pd.DataFrame(grid_rf.grid_scores_), 'GridResults/blender_rf.csv', index=False)
pd.DataFrame.to_csv(pd.DataFrame(grid_naive.grid_scores_), 'GridResults/blender_naive.csv', index=False)
pd.DataFrame.to_csv(pd.DataFrame(grid_xgb.grid_scores_), 'GridResults/blender_xgb.csv', index=False)

TODO: 
- Make submission by using each individual layer1 model
    - Try to add more non-correlated predictions for layer1
- Make submission by using single blender model
- Make submission by using average of multiple blender model
    - Try to add more non-correlated blender models
- Make prediction by using layer1 predictions only. 

- Grid search for blender by using predictions only
- Grid search for blender by using predictions and basic stats only


Although these results looks promising, I bet, that there is a trick with feature engineering. 

As the next step, try to do similarity thing, and go through my notes to get some ideas. 


## Grid search for blender model with predictions only

In [2]:
train_l1_holdout_blend_v1 = pd.read_csv('Blend_data/Blend_v1_train_l1_holdout.csv')
train_y_l1_holdout = pd.read_csv('Blend_data/Blend_v1_train_y_holdout.csv')

test_blend_v1 = pd.read_csv('Blend_data/Blend_v1_test.csv')

In [9]:
train_l1_holdout_blend_v1 = train_l1_holdout_blend_v1.iloc[:,-6:]

In [12]:
# Determine model parameters: 
params_logreg = {'n_jobs':[4],
                 'C':[0.0001,0.1,0.5,1.5,2,5,10], 
                 'fit_intercept':[False,True], 
                 'max_iter':[10,100,500,1000,2000],
                 'solver':['newton-cg','lbfgs','liblinear','sag'],
                 'tol':[0.00001,0.0001,0.001,0.1,0.5],
                 'verbose':[0],
                 'random_state':[42]}

params_knn = {'n_jobs':[4],
              'n_neighbors':[5,50,100,500,1000,1500,2000,3000,4000,5000], 
              'p':[1,2,3,4,5,10,11,12,15,20],
              'leaf_size':[10,20,30,40,50,60],
              'algorithm':['auto','ball_tree','kd_tree','brute']}

# care for max_features
params_rf = {'n_jobs':[4], 
             'criterion':['gini', 'entropy'],
             'n_estimators':[250,500,1000,1500,2000,3000,4000,5000], 
             'max_features':[1,2,3,4,5,6], 
             'max_depth':[1,5,10,15,20,25,50,100], 
             'min_samples_split':[1,5,10,25,50], 
             'min_samples_leaf':[1,5,10,25,50], 
             'oob_score':[True,False], 
             'verbose':[0], 
             'random_state':[42]}

params_xgb = {'silent':[1],
              'nthread':[4], 
              'seed':[42], 
              'max_depth':[1,5,10,15,20,25],
              'subsample':[0.2,0.5,0.7,1],
              'reg_lambda':[1,2,5,10],
              'learning_rate':[0.0001,0.001,0.01,0.1,0.3], 
              'gamma':[0,0.0001,0.001,0.01,0.1],
              'n_estimators':[250,500,1000,1500,2000,3000,4000,5000]
             }

params_naive = {'alpha':[0.1,0.5,0.75,1,1.5,2,5,10,15,25,30,40,50,60,70,80,90,100,200,500], 
                'binarize':[0.1,0.5,0.75,1,1.5,2,5,10,15,25,30,40,50,60,70,80,90,100,200,500],
                'fit_prior':[True, False], 
               }

params_ada = {'n_estimators':[250,500,1000,1500,2000,3000,4000,5000],
              'learning_rate':[0.0001,0.001,0.01,0.1,0.5,0.75,1.5,2,5,10,25,50], 
              'random_state':[42] 
             }

# Determine models
model_logreg = LogisticRegression()
model_knn = KNeighborsClassifier()
model_rf = RandomForestClassifier()
model_xgb = xgb.XGBClassifier()
model_naive = BernoulliNB()
model_ada = AdaBoostClassifier()

In [13]:
#Logistic regression
grid_logreg = RandomizedSearchCV(n_iter = 500, estimator=model_logreg, param_distributions=params_logreg, 
                                 n_jobs=2, cv=5, refit=True, verbose=1, scoring='log_loss')

grid_logreg.fit(train_l1_holdout_blend_v1, train_y_l1_holdout.target)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    5.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   22.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:   49.3s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:  1.5min
[Parallel(n_jobs=2)]: Done 1246 tasks      | elapsed:  2.3min
[Parallel(n_jobs=2)]: Done 1796 tasks      | elapsed:  3.5min
[Parallel(n_jobs=2)]: Done 2446 tasks      | elapsed:  4.8min
[Parallel(n_jobs=2)]: Done 2500 out of 2500 | elapsed:  4.9min finished
/home/ngergoo/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:1207: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  for (class_, warm_start_coef_) in zip(classes_, warm_start_coef))
/home/ngergoo/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:1207: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  for (class_, warm_start_coef_) in zip(classes_, warm_start_coef))
/home/ng

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=500, n_jobs=2,
          param_distributions={'C': [0.0001, 0.1, 0.5, 1.5, 2, 5, 10], 'n_jobs': [4], 'verbose': [0], 'tol': [1e-05, 0.0001, 0.001, 0.1, 0.5], 'fit_intercept': [False, True], 'random_state': [42], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag'], 'max_iter': [10, 100, 500, 1000, 2000]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='log_loss', verbose=1)

In [14]:
# Knn
grid_knn = RandomizedSearchCV(n_iter=10, estimator=model_knn, param_distributions=params_knn, n_jobs=2, cv=5, 
                             refit=True, verbose=1, scoring='log_loss')

grid_knn.fit(train_l1_holdout_blend_v1, train_y_l1_holdout.target)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  5.6min
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:  6.5min finished
/home/ngergoo/anaconda2/lib/python2.7/site-packages/sklearn/metrics/pairwise.py:1060: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  for s in gen_even_slices(Y.shape[0], n_jobs))
/home/ngergoo/anaconda2/lib/python2.7/site-packages/sklearn/metrics/pairwise.py:1060: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  for s in gen_even_slices(Y.shape[0], n_jobs))


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
          fit_params={}, iid=True, n_iter=10, n_jobs=2,
          param_distributions={'n_neighbors': [5, 50, 100, 500, 1000, 1500, 2000, 3000, 4000, 5000], 'n_jobs': [4], 'leaf_size': [10, 20, 30, 40, 50, 60], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 'p': [1, 2, 3, 4, 5, 10, 11, 12, 15, 20]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='log_loss', verbose=1)

In [15]:
# Random Forest
grid_rf = RandomizedSearchCV(estimator=model_rf, param_distributions=params_rf, n_jobs=2, cv=5, refit=True, 
                             verbose=1, scoring='log_loss')

grid_rf.fit(train_l1_holdout_blend_v1, train_y_l1_holdout.target)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 16.8min
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed: 17.0min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params={}, iid=True, n_iter=10, n_jobs=2,
          param_distributions={'n_estimators': [250, 500, 1000, 1500, 2000, 3000, 4000, 5000], 'min_samples_split': [1, 5, 10, 25, 50], 'oob_score': [True, False], 'n_jobs': [4], 'criterion': ['gini', 'entropy'], 'verbose': [0], 'max_features': [1, 2, 3, 4, 5, 6], 'random_state': [42], 'max_depth': [1, 5, 10, 15, 20, 25, 50, 100], 'min_samples_leaf': [1, 5, 10, 25, 50]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='log_loss', verbose=1)

In [16]:
# Bernoulli Bayes
grid_naive = RandomizedSearchCV(n_iter=40, estimator=model_naive, param_distributions=params_naive, n_jobs=8, 
                                cv=5, refit=True, verbose=1, scoring='log_loss')
grid_naive.fit(train_l1_holdout_blend_v1, train_y_l1_holdout.target)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=8)]: Done  88 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    1.7s finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True),
          fit_params={}, iid=True, n_iter=40, n_jobs=8,
          param_distributions={'binarize': [0.1, 0.5, 0.75, 1, 1.5, 2, 5, 10, 15, 25, 30, 40, 50, 60, 70, 80, 90, 100, 200, 500], 'alpha': [0.1, 0.5, 0.75, 1, 1.5, 2, 5, 10, 15, 25, 30, 40, 50, 60, 70, 80, 90, 100, 200, 500], 'fit_prior': [True, False]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='log_loss', verbose=1)

In [17]:
# XGB
splitter = StratifiedShuffleSplit(y=train_y_l1_holdout, n_iter=1, train_size=0.7, 
                                      test_size=0.3, random_state=42)

for train_index, test_index in splitter: 
    train_1 = train_l1_holdout_blend_v1.iloc[train_index,:]
    train_1.reset_index(drop=True, inplace=True)
    train_y_1 = train_y_l1_holdout.iloc[train_index,:]
    train_y_1.reset_index(drop=True, inplace=True)
    train_2 = train_l1_holdout_blend_v1.iloc[test_index,:]
    train_2.reset_index(drop=True, inplace=True)
    train_y_2 = train_y_l1_holdout.iloc[test_index,:]
    train_y_2.reset_index(drop=True, inplace=True)


xgb_fit_params = {'eval_set':[(train_1, train_y_1.target),(train_2, train_y_2.target)], 
                  'early_stopping_rounds':25, 'verbose':0}

grid_xgb = RandomizedSearchCV(n_iter=20, estimator=model_xgb, param_distributions=params_xgb, 
                              n_jobs=2, cv=5, refit=True, verbose=1, scoring='log_loss', 
                              fit_params=xgb_fit_params
                             )

grid_xgb.fit(train_l1_holdout_blend_v1, train_y_l1_holdout.target)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  3.1min
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:  4.9min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
          fit_params={'eval_set': [(       pred_logreg  pred_knn   pred_rf  pred_naive  pred_ada  pred_xgb
0         0.520858   0.52000  0.535538    0.504673  0.505963  0.524817
1         0.478932   0.51325  0.504731    0.504673  0.512173  0.490121
2         0.539671   0.52250  0.521396    0.504673  0.512173 ...    1
11557    1
11558    0
Name: target, dtype: int64)], 'early_stopping_rounds': 25, 'verbose': 0},
          iid=True, n_iter=20, n_jobs=2,
          param_distributions={'n_estimators': [250, 500, 1000, 1500, 2000, 3000, 4000, 5000], 'subsample': [0.2, 0.5, 0.7, 1], 'reg_lambda': [1,

In [ ]:
# Adaboost - gives back some weird value error, about nans/infinite values in df X?!
grid_ada = RandomizedSearchCV(n_iter=10, estimator=model_ada, param_distributions=params_ada, n_jobs=8, 
                              cv=5, refit=True, verbose=1, scoring='log_loss')
grid_ada.fit(train_l1_holdout_blend_v1, train_y_l1_holdout.target)

In [19]:
# results
print 'Logreg: ', grid_logreg.best_params_, grid_logreg.best_score_
print 'KNN: ', grid_knn.best_params_, grid_knn.best_score_
print 'RandomForest: ', grid_rf.best_params_, grid_rf.best_score_
print 'Bernoulli bayes: ', grid_naive.best_params_, grid_naive.best_score_
# print 'Adaboost: ', grid_ada.best_params_, grid_ada.best_score_
print 'Xgboost: ', grid_xgb.best_params_, grid_xgb.best_score_

Logreg:  {'fit_intercept': True, 'C': 1.5, 'n_jobs': 4, 'verbose': 0, 'solver': 'lbfgs', 'max_iter': 500, 'random_state': 42, 'tol': 0.0001} -0.691378634761
KNN:  {'p': 2, 'n_jobs': 4, 'leaf_size': 50, 'algorithm': 'ball_tree', 'n_neighbors': 4000} -0.69173965538
RandomForest:  {'oob_score': False, 'n_jobs': 4, 'verbose': 0, 'min_samples_leaf': 1, 'n_estimators': 3000, 'max_features': 1, 'random_state': 42, 'criterion': 'gini', 'min_samples_split': 1, 'max_depth': 5} -0.691632767865
Bernoulli bayes:  {'binarize': 0.75, 'alpha': 0.5, 'fit_prior': True} -0.693093824732
Xgboost:  {'silent': 1, 'learning_rate': 0.01, 'nthread': 4, 'n_estimators': 2000, 'subsample': 0.7, 'reg_lambda': 10, 'seed': 42, 'max_depth': 5, 'gamma': 0} -0.691921694662


In [20]:
pd.DataFrame.to_csv(pd.DataFrame(grid_logreg.grid_scores_), 'GridResults/blender_predonly_logreg.csv', 
                    index=False)
pd.DataFrame.to_csv(pd.DataFrame(grid_knn.grid_scores_), 'GridResults/blender_predonly_knn.csv', index=False)
pd.DataFrame.to_csv(pd.DataFrame(grid_rf.grid_scores_), 'GridResults/blender_predonly_rf.csv', index=False)
pd.DataFrame.to_csv(pd.DataFrame(grid_naive.grid_scores_), 'GridResults/blender_predonly_naive.csv', 
                    index=False)
pd.DataFrame.to_csv(pd.DataFrame(grid_xgb.grid_scores_), 'GridResults/blender_predonly_xgb.csv', index=False)